In [1]:
import re
import pandas as pd

In [2]:
df = pd.read_csv('general_and_score.csv')
df.head()

,ФИО,Пол,Роль в мероприятии,Список компетенций,Должность,Категория,Начало трудового стажа,Место работы,Профессия,Дата рождения,Начало трудовой деятельности в РОСАТОМ,Образование,Место образования,Год оканчания,Специальность,"Баллы, ед.","Баллы, %",Баллы по ключевым навыкам,labels
0,ФИО_0,0,Участник мероприятия,Цифровое ПСР-Предприятие (Lean Smart Plant);,Инженер по подготовке производства,Инженер,1993-06-01,АО Чепецкий механический завод,Промышленная автоматика,1975-04-14,2001-01-25,NaN,NaN,NaN,NaN,106,63.90,60.28,3.0
1,ФИО_1,1,Участник мероприятия,Управление качеством;,Главный специалист,NaN,2000-08-01,Акционерное общество Научно-исследовательский...,NaN,1977-07-19,2021-09-14,NaN,NaN,NaN,NaN,100,19.18,19.18,10.0
2,ФИО_2,0,Участник мероприятия,Сварочные технологии;,Электрогазосварщик,NaN,NaN,"""Филиал Акционерного общества """"Русатом Инфрас...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,13.78,13.78,10.0
3,ФИО_3,0,Участник мероприятия,Инженер-конструктор;,инженер-конструктор,NaN,июл.05,"""Акционерное Общество """"Машиностроительный зав...",NaN,23.06.1986,NaN,NaN,NaN,NaN,NaN,100,38.48,38.48,3.0
4,ФИО_4,0,Участник мероприятия,Геодезия;,Мастер-лаборант,NaN,NaN,ОБПОУ Курский монтажный техникум,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,48.00,48.0,3.0


In [5]:
df.columns = ['name', 'gender', 'event_role', 'competence_list', 'job_title', 'category', 'start_work_dt',
              'job_place', 'profession', 'birth_date', 'rosatom_start_dt', 'education', 'education_place',
              'release_year', 'speciality', 'points', 'points_pct', 'spec_points', 'labels']

In [6]:
# add id column
df.reset_index(names='id', inplace=True)

In [7]:
# add gender name
df['gender_name'] = df['gender'].apply(lambda x: 'мужчина' if x == 1.0 else 'женщина')

### Year, age and experience extraction

In [8]:
def extract_year(col: pd.Series):
    col = str(col)
    
    # for 1985-05-19
    year = re.search(r'\d{4}\-\d{2}\-\d{2}', col)
    if year:
        return year.group(0).split('-')[0]
    # for 09.07.1993
    year = re.search(r'\d{2}\.\d{2}\.\d{4}', col)
    if year:
        return year.group(0).split('.')[-1]
    # for 02.2014
    year = re.search(r'\d{2}\.\d{4}', col)
    if year:
        return year.group(0).split('.')[-1]
    # for 2013
    year = re.search(r'\d{4}', col) # if just a YEAR
    if year:
        return col
    # for ноя.13
    year = re.search(r'\w{3}\.\d{2}', col)
    if year:
        return '20' + year.group(0).split('.')[-1]
    

df['birth_year'] = df['birth_date'].apply(extract_year)
df['start_work_year'] = df['start_work_dt'].apply(extract_year)
df['rosatom_start_year'] = df['rosatom_start_dt'].apply(extract_year)

In [9]:
df['birth_year'] = pd.to_numeric(df['birth_year'], errors='coerce')
df['start_work_year'] = pd.to_numeric(df['start_work_year'], errors='coerce')
df['rosatom_start_year'] = pd.to_numeric(df['rosatom_start_year'], errors='coerce')

In [10]:
df['age'] = 2022 - df['birth_year']
df['experience'] = 2022 - df['start_work_year']
df['rosatom_experience'] = 2022 - df['rosatom_start_year']

In [11]:
def exp_cluster(col: pd.Series):
    if col < 1:
        return 'Без опыта'
    elif 1 <= col < 3:
        return '1 - 3 года'
    elif 3 <= col < 6:
        return '3 - 6 года'
    elif 6 <= col:
        return 'более 6 лет'
    else:
        return None
    
def birth_cluster(col: pd.Series):
    if col < 18:
        return 'младше 18 лет'
    elif 18 <= col < 25:
        return '18 - 25 лет'
    elif 25 <= col < 32:
        return '25 - 32 лет'
    elif 32 <= col < 40:
        return '32 - 40 лет'
    elif col >= 40:
        return 'старше 40 лет'
    else:
        return None
    

df['bd_cluster'] = df['age'].apply(birth_cluster)
df['experience_cluster'] = df['experience'].apply(exp_cluster)
df['rosatom_experience_cluster'] = df['rosatom_experience'].apply(exp_cluster)

### extract company type and name

In [12]:
def clean_text(text: str):
    text = str(text)

    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub("[\"\']", " ", text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    return text

def get_comp_forms(text: str):
    text = clean_text(text)
    if any(w in text for w in ['пао', 'публичное акционерное общество']):
        return 'ПАО'
    elif any(w in text for w in ['ао', 'акционерное общество', 'филиал акционерного общества']):
        return 'АО'
    elif any(w in text for w in ['фгуп', 'федеральное государственное унитарное предприятие']):
        return 'ФГУП'
    elif any(w in text for w in ['оао', 'открытое акционерное общество']):
        return 'ОАО'
    elif any(w in text for w in ['ооо', 'общество с ограниченной ответственностью']):
        return 'ООО'
    else:
        return 'Неизвестно'
    
def get_comp_name(text: str):
    text = clean_text(text)
    forms = ['пао', 'публичное акционерное общество', 'ао', 'акционерное общество', 'филиал акционерного общества',
            'фгуп', 'федеральное государственное унитарное предприятие',
            'оао', 'открытое акционерное общество', 'ооо', 'общество с ограниченной ответственностью']
    text = re.sub("|".join(forms), "", text)
    text = re.sub(' +', ' ', text)  # remove multi spaces
    return text.strip().upper()


df['company_form'] = df['job_place'].apply(get_comp_forms)
df['company_name'] = df['job_place'].apply(get_comp_name)

/tmp/ipykernel_277658/5543683.py:8: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)


In [13]:
df.head()

,id,name,gender,event_role,competence_list,job_title,category,start_work_dt,job_place,profession,...,birth_year,rosatom_start_year,age,experience,rosatom_experience,bd_cluster,experience_cluster,rosatom_experience_cluster,company_form,company_name
0,0,ФИО_0,0,Участник мероприятия,Цифровое ПСР-Предприятие (Lean Smart Plant);,Инженер по подготовке производства,Инженер,1993-06-01,АО Чепецкий механический завод,Промышленная автоматика,...,1975.0,2001.0,47.0,29.0,21.0,старше 40 лет,более 6 лет,более 6 лет,АО,ЧЕПЕЦКИЙ МЕХАНИЧЕСКИЙ ЗАВОД
1,1,ФИО_1,1,Участник мероприятия,Управление качеством;,Главный специалист,NaN,2000-08-01,Акционерное общество Научно-исследовательский...,NaN,...,1977.0,2021.0,45.0,22.0,1.0,старше 40 лет,более 6 лет,1 - 3 года,АО,НАУЧНОИССЛЕДОВАТЕЛЬСКИЙ ИНСТИТУТ НАУЧНОПРОИЗВО...
2,2,ФИО_2,0,Участник мероприятия,Сварочные технологии;,Электрогазосварщик,NaN,NaN,"""Филиал Акционерного общества """"Русатом Инфрас...",NaN,...,NaN,NaN,NaN,NaN,NaN,None,None,None,АО,РУСАТОМ ИНФРАСТРУКТУРНЫЕ РЕШЕНИЯ В ГОРОДЕ НОВО...
3,3,ФИО_3,0,Участник мероприятия,Инженер-конструктор;,инженер-конструктор,NaN,июл.05,"""Акционерное Общество """"Машиностроительный зав...",NaN,...,1986.0,NaN,36.0,17.0,NaN,32 - 40 лет,более 6 лет,None,АО,МАШИНОСТРОИТЕЛЬНЫЙ ЗАВОД
4,4,ФИО_4,0,Участник мероприятия,Геодезия;,Мастер-лаборант,NaN,NaN,ОБПОУ Курский монтажный техникум,NaN,...,NaN,NaN,NaN,NaN,NaN,None,None,None,Неизвестно,ОБПОУ КУРСКИЙ МОНТАЖНЫЙ ТЕХНИКУМ


In [14]:
ng = pd.read_csv('name_group.csv')
ng.head()

,ФИО,Компетенция,feat_0,score,n_users,in_group
0,ФИО_34,Электромонтаж,7.034772,0.957542,1,индивидуальный
1,ФИО_61,Электромонтаж,-2.378477,-0.329388,1,индивидуальный
2,ФИО_93,Электромонтаж,-5.115188,-0.787578,1,индивидуальный
3,ФИО_116,Электромонтаж,16.330892,2.216796,1,индивидуальный
4,ФИО_155,Электромонтаж,-1.332281,-0.024953,1,индивидуальный


In [35]:
full = df.merge(right=ng, left_on='name', right_on='ФИО', how='left')
full.drop(['ФИО', 'n_users'], axis=1, inplace=True)

In [36]:
full.head()

,id,name,gender,event_role,competence_list,job_title,category,start_work_dt,job_place,profession,...,rosatom_experience,bd_cluster,experience_cluster,rosatom_experience_cluster,company_form,company_name,Компетенция,feat_0,score,in_group
0,0,ФИО_0,0,Участник мероприятия,Цифровое ПСР-Предприятие (Lean Smart Plant);,Инженер по подготовке производства,Инженер,1993-06-01,АО Чепецкий механический завод,Промышленная автоматика,...,21.0,старше 40 лет,более 6 лет,более 6 лет,АО,ЧЕПЕЦКИЙ МЕХАНИЧЕСКИЙ ЗАВОД,Цифровое ПСР-Предприятие (Lean Smart Plant),-0.689026,0.601190,командный
1,1,ФИО_1,1,Участник мероприятия,Управление качеством;,Главный специалист,NaN,2000-08-01,Акционерное общество Научно-исследовательский...,NaN,...,1.0,старше 40 лет,более 6 лет,1 - 3 года,АО,НАУЧНОИССЛЕДОВАТЕЛЬСКИЙ ИНСТИТУТ НАУЧНОПРОИЗВО...,Управление качеством,11.951287,-1.650741,командный
2,2,ФИО_2,0,Участник мероприятия,Сварочные технологии;,Электрогазосварщик,NaN,NaN,"""Филиал Акционерного общества """"Русатом Инфрас...",NaN,...,NaN,None,None,None,АО,РУСАТОМ ИНФРАСТРУКТУРНЫЕ РЕШЕНИЯ В ГОРОДЕ НОВО...,Сварочные технологии,8.332544,-1.473122,индивидуальный
3,3,ФИО_3,0,Участник мероприятия,Инженер-конструктор;,инженер-конструктор,NaN,июл.05,"""Акционерное Общество """"Машиностроительный зав...",NaN,...,NaN,32 - 40 лет,более 6 лет,None,АО,МАШИНОСТРОИТЕЛЬНЫЙ ЗАВОД,Инженер-конструктор,5.269416,0.537858,индивидуальный
4,4,ФИО_4,0,Участник мероприятия,Геодезия;,Мастер-лаборант,NaN,NaN,ОБПОУ Курский монтажный техникум,NaN,...,NaN,None,None,None,Неизвестно,ОБПОУ КУРСКИЙ МОНТАЖНЫЙ ТЕХНИКУМ,Геодезия,-5.812724,-0.725446,командный


In [17]:
full.to_csv('final.csv', index=False)

In [37]:
jn = pd.read_csv('job_names3.csv')
jn.head()

,name_id,company_name,division,city
0,40,NaN,NaN,NaN
1,80,AKKUYU NÜKLEER ANONİM ŞİRKETİ,NaN,NaN
2,19,NAN,NaN,NaN
3,325,TİTAN IC İÇTAŞ İNŞAAT AŞ,NaN,NaN
4,179,АККУЮ НУКЛЕАР,NaN,NaN


In [38]:
full2 = full.merge(jn, left_on='company_name', right_on='company_name', how='left')

In [39]:
full2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 713 entries, 0 to 712
Data columns (total 39 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          713 non-null    int64  
 1   name                        713 non-null    object 
 2   gender                      713 non-null    int64  
 3   event_role                  713 non-null    object 
 4   competence_list             713 non-null    object 
 5   job_title                   635 non-null    object 
 6   category                    210 non-null    object 
 7   start_work_dt               565 non-null    object 
 8   job_place                   707 non-null    object 
 9   profession                  178 non-null    object 
 10  birth_date                  505 non-null    object 
 11  rosatom_start_dt            390 non-null    object 
 12  education                   143 non-null    object 
 13  education_place             143 non

In [33]:
full2.to_csv('final2.csv', index=False)